# Programmation dynamique et mémoïsation

# 1. Problème du sujet 0 : 
Trouver la somme maximum qu'on peut atteindre en traversant les cellules d'un tableau d'entiers à 2 dimensions en n'autorisant que des déplacements vers la droite ou vers le bas. La cellule de départ est le coin supérieur gauche, et la cellule d'arrivée est le coin inférieur droit.

In [ ]:
T = [[4, 1, 1, 3],
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

Parcours idéal de l'exemple : 4 - 2 - 3 - 1 - 5 - 1 : somme = 16

## Solution récursive :
Solution récurisve naive telle que proposée dans le sujet 0 de NSI.

In [ ]:
def somme_max(T, i, j):
    if i == j == 0:
        return T[i][j]
    elif i == 0:
        return T[0][j] + somme_max(T, 0, j-1)
    elif j == 0:
        return T[i][0] + somme_max(T, i-1, 0)
    else:
        return T[i][j] + max(somme_max(T, i-1, j), somme_max(T, i, j-1))

In [ ]:
somme_max(T, 2, 3)

**Problème** : on *re-calcule beaucoup de choses* ! **Très lourd en mémoire et la pile de récursion risque vite d'exploser**.

## Approche récursive avec mémoïsation (bottom-down)
L'idée est de **stocker dans un tableau annexe toutes les valeurs déjà calculées** une fois pour ne pas recommencer !

**On conserve l'approche descendante de la récursivité** : la solution globale fait appel au fur et à mesure au solutions des sous-problèmes (en vérifiant si une valeur n'a pas déjà été mémorisée avant de se lancer dans le calcul récursif).

In [4]:
def somme_max_memo(T, i, j):
    def sm(T, i, j):
        if memoire[i][j] is not None:
            return memoire[i][j]
        elif i == 0:
            memoire[0][j] = T[0][j] + sm(T, 0, j-1)
        elif j == 0:
            memoire[i][0] = T[i][0] + sm(T, i-1, 0)
        else:
            gauche, haut = sm(T, i, j-1), sm(T, i-1, j)
            memoire[i][j] = T[i][j] + max(gauche, haut)
        return memoire[i][j]
   
    memoire = [[None for _ in range(len(T[0]))] for _ in range(len(T))]
    memoire[0][0] =  T[0][0]
    return sm(T, i, j)

In [5]:
somme_max_memo(T, 2, 3)

16

## Approche par la programmation dynamique
L'**approche ascendante de la programmation dynamique** consiste à construire **de façon itérative** les **solutions des sous-problèmes** pour atteindre la solution globale qui combine ces sous-problèmes.

In [6]:
def somme_max_dyn(T, i, j):    
    memoire = [[T[i][j] for j in range(len(T[0]))] for i in range(len(T))]
    for j in range(1, len(T[0])):
        memoire[0][j] = T[0][j] + memoire[0][j-1]
    for i in range(1, len(T)):
        memoire[i][0] = T[i][0] + memoire[i-1][0]
    for i in range(1, len(T)):
        for j in range(1, len(T[0])):
            memoire[i][j] = T[i][j] + max(memoire[i-1][j], memoire[i][j-1])
    return memoire[i][j]

In [7]:
somme_max_dyn(T, 2, 3)

16

## Simplification possible : optimisation de la mémoire
Il est possible de ne conserver la mémoire que d'une ligne à la fois.

In [8]:
def somme_max_dyn_opt(T, i, j):    
    ligne = [T[0][j] for j in range(len(T[0]))]
    for j in range(1, len(T[0])):
        ligne[j] = T[0][j] + ligne[j-1]
    for i in range(1, len(T)):
        ligne[0] = ligne[0] + T[i][0]
        for j in range(1, len(T[0])):
            ligne[j] = T[i][j] + max(ligne[j-1], ligne[j])
    return ligne[-1]

In [9]:
somme_max_dyn_opt(T, 2, 3)

16

# Voir la durée pour un tableau plus grand.
On propose un tableau de 10 lignes et 20 colonnes, de nombre aléaoires entre 1 et 9.

In [10]:
from random import randint

In [11]:
i, j = 10, 20
T = [[randint(1, 9) for _ in range(j)] for _ in range(i)]

for n in range(len(T)):
    for m in range(len(T[0])):
        print(T[n][m], end = '  ')
    print()

4  5  5  5  8  4  4  9  3  1  5  7  6  9  3  5  9  5  9  8  
9  7  2  5  5  7  4  3  3  5  2  3  7  9  3  4  5  5  3  4  
3  4  6  1  9  2  7  8  5  3  7  5  2  7  4  1  2  6  1  8  
5  4  9  2  6  2  8  8  6  1  7  9  4  2  2  3  7  1  5  4  
1  6  2  9  9  3  5  3  2  2  7  1  3  8  1  6  8  8  8  7  
8  7  7  6  1  2  9  1  6  6  3  1  4  1  7  9  1  2  8  1  
1  7  4  3  6  6  9  8  4  5  8  4  7  3  7  7  5  3  9  1  
3  1  7  9  6  1  1  2  4  2  3  6  5  1  8  1  6  2  1  5  
6  5  4  3  3  4  8  1  9  7  6  9  4  2  3  2  6  2  3  2  
6  3  7  8  6  8  6  7  7  3  5  7  2  2  7  8  5  8  3  1  


In [12]:
somme_max_dyn(T, i-1, j-1)

175

In [13]:
somme_max_dyn_opt(T, i-1, j-1)

175

In [14]:
somme_max_memo(T, i-1, j-1)

175

In [15]:
somme_max(T, i-1, j-1) # on sent passer le temps...

175

# Complément : Algo glouton
On propose une solution par algorithme glouton : pas forcément optimale !
Le critère local d'optimisation retenu est de s'orienter vers la cellule la plus "grosse" à chaque carrefour.

In [29]:
def somme_max_glouton(T, i, j):
    s = T[0][0]
    n, m = 0, 0
    while (n != len(T) - 1) or (m != len(T[0]) - 1):
        if m == len(T[0]) - 1:
            n += 1
        elif n == len(T) - 1:
            m += 1
        elif T[n][m+1] > T[n+1][m]:
            m += 1
        else:
            n += 1
        s += T[n][m]
    return s

## Cas où l'algo glouton fonctionne (coup de bol !)

In [30]:
T = [[4, 1, 1, 3],
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

i, j = len(T), len(T[0])
print('glouton :', somme_max_glouton(T, i-1, j-1))
print('solution optimale :', somme_max_dyn_opt(T, i-1, j-1))

glouton : 16
solution optimale : 16


## Cas où l'algo glouton ne fonctionne pas (ben c'est pas étonnant qu'il y ait des échecs...)

In [31]:
T = [[4, 1, 1000, 3], # on va "rater" le 1000
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

i, j = len(T), len(T[0])
print('glouton :', somme_max_glouton(T, i-1, j-1))
print('solution optimale :', somme_max_dyn_opt(T, i-1, j-1))

glouton : 16
solution optimale : 1013


## Cas aléatoire avec un tableau assez grand :
On compare la solution de l'algo glouton avec un algorithme où on navigue aléateoirement dans le tableau pour vérifier si l'algo glouton tend plutôt à donner une solution "intéressante".

In [32]:
from random import choice
def pile_ou_face():
    """ Une fonction de pile ou face à moitié inutile mais qui m'amuse... """
    return choice(['Pile', 'Face'])

In [33]:
def somme_alea(T, i, j):
    s = T[0][0]
    n, m = 0, 0
    while (n != len(T) - 1) or (m != len(T[0]) - 1):
        if m == len(T[0]) - 1:
            n += 1
        elif n == len(T) - 1:
            m += 1
        else:
            if pile_ou_face() == 'Pile':
                n += 1
            else:
                m += 1
        s += T[n][m]
    return s

### Tableau de 1000 lignes et 1000 colonnes :
On constate que l'algo glouton apporte quand même un intérêt par raport à la marche aléatoire, mais il n'atteint pas la solution optimale.

In [34]:
i, j = 1000, 1000
T = [[randint(1, 9) for _ in range(j)] for _ in range(i)]
    
print(somme_alea(T, i-1, j-1))
print(somme_max_glouton(T, i-1, j-1))
print(somme_max_dyn_opt(T, i-1, j-1))
print(somme_max_memo(T, i-1, j-1))

9837
12480
14321
14321


In [35]:
%timeit somme_max_dyn_opt(T, i-1, j-1)

300 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit somme_max_dyn(T, i-1, j-1)

431 ms ± 746 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit somme_max_memo(T, i-1, j-1)

673 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Recherche du chemin
On s'appuie sur l'algo dynamique (sans optimiser la mémoire)

In [53]:
def chemin_max_dyn(T, i, j):    
    memoire = [[T[i][j] for j in range(len(T[0]))] for i in range(len(T))]
    chemin = [(0, 0)]
    for j in range(1, len(T[0])):
        memoire[0][j] = T[0][j] + memoire[0][j-1]
    for i in range(1, len(T)):
        memoire[i][0] = T[i][0] + memoire[i-1][0]
    for i in range(1, len(T)):
        for j in range(1, len(T[0])):
            memoire[i][j] = T[i][j] + max(memoire[i-1][j], memoire[i][j-1])
            
            
    n, m = len(memoire) - 1, len(memoire[0]) - 1
    cellules = [(n, m)]
    nombres = [T[n][m]]
    sommes = [memoire[n][m]]
    while (n != 0) or (m != 0):
        if m == 0:
            n -= 1
        elif n == 0:
            m -= 1
        elif memoire[n][m-1] > memoire[n-1][m]:
            m -= 1
        else:
            n -= 1
        cellules.insert(0, (n, m))
        sommes.insert(0, memoire[n][m])
        nombres.insert(0, T[n][m])
        
    return cellules, nombres, sommes

In [54]:
T = [[4, 1, 1, 3],
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

chemin_max_dyn(T, 2, 3)

([(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3)],
 [4, 2, 3, 1, 5, 1],
 [4, 6, 9, 10, 15, 16])

In [107]:
i, j = 20, 20
T = [[randint(-99, 99) for _ in range(j)] for _ in range(i)]

for n in range(len(T)):
    for m in range(len(T[0])):
        print(str(T[n][m]).center(3), end = '  ')
    print()
    
cellules, nombres, sommes = chemin_max_dyn(T, i-1, j-1) 
#print(cellules)
#print(nombres)
#print(sommes)

print()
cellules = set(cellules)
for n in range(len(T)):
    for m in range(len(T[0])):
        if (n, m) in cellules:
            print(str(T[n][m]).center(4, ' '), end = ' ')
        else:
            print('____', end = ' ')
    print()

print()
k = 0
for n in range(len(T)):
    for m in range(len(T[0])):
        if (n, m) in cellules:
            print(str(sommes[k]).center(4, ' '), end = ' ')
            k += 1
        else:
            print('____', end = ' ')
    print()

-16   71   44  -67  -72   58  -95   -6  -31  -21   6    13  -10   43   20  -94  -89   8    37  -12  
-70   48   58   94   34  -14   34   45   91   53   39   23  -35  -56   30   83  -68  -17  -30   95  
-78  -56  -45   37   33   72   95   5    25   42   90   50   13  -34  -28   53  -83   33  -19  -37  
 25  -58  -39   75   29   39  -19  -80   49   66   8    11   -8  -78  -94   62  -13  -59   34   96  
-28  -47  -84   74   -7  -82  -33   65  -68   48   65   -4  -94  -95  -87  -13   76  -57  -57  -44  
-91   13  -78  -69   -8   72   68   -1   38  -62   52  -37  -76   28  -22  -88  -99   45  -69   62  
-69  -45   17   77  -89   30   48   73  -52  -69  -85   81  -57   90  -79   58   12  -50  -52  -78  
-20   78  -97  -83  -29  -60   14  -81  -34  -62  -71  -58   7   -44  -87   56  -18   14   90  -12  
-46  -12   57  -51  -75  -55   79  -94  -70  -78  -30   29   21   20  -26  -33  -87   87  -78  -38  
 82   5    92   37   32   37   47   81   40  -11  -40   97  -19   24   43  -56   0    65   